In [1]:
import sys
import pathlib

sys.path.append(r"C:\Users\amrul\programming\deep_learning\dl_projects\Generative_Deep_Learning_2nd_Edition")

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
from torchvision import datasets, transforms
from tqdm import tqdm

IMAGE_SIZE = 64
BATCH_SIZE = 64
DATASET_REPETITIONS = 5
LOAD_MODEL = False

NOISE_EMBEDDING_SIZE = 32
PLOT_DIFFUSION_STEPS = 20

# optimization
EMA = 0.999
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-4
EPOCHS = 50


In [3]:
from notebooks.utils import display

In [4]:
import torch
import torch.nn as nn
from ddm_torch_model import DiffusionModel,get_flower_images_train_dataset,convert_images_torch_to_numpy_for_display,training_loop,Normalizer

In [5]:
train_data = get_flower_images_train_dataset()
print(f"Loaded flower train dataset : {len(train_data)}")

Loaded flower train dataset : 6552


In [6]:
image,label=train_data[0]
print(image.shape,type(image))

torch.Size([3, 64, 64]) <class 'torch.Tensor'>


In [7]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_data,batch_size=BATCH_SIZE,shuffle=True)

In [8]:
from ddm_torch_model import calc_mean_and_std
recalculate_means_stds=False

if recalculate_means_stds:
    total_mean = torch.zeros(3)
    total_var = torch.zeros(3)
    for images,_ in tqdm(train_loader):
        mean,std = calc_mean_and_std(images)
        total_mean += mean
        total_var += std**2

    final_mean =total_mean/len(train_loader)
    final_var = total_var/len(train_loader)
    final_std = torch.sqrt(final_var)
    print(f"final mean : {final_mean}, final std : {final_std}")
else:
    final_mean = torch.tensor([0.4352, 0.3771, 0.2871])
    final_std = torch.tensor([0.2853, 0.2328, 0.2580])

In [9]:
final_std

tensor([0.2853, 0.2328, 0.2580])

In [10]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [11]:
normalizer = Normalizer()
normalizer.mean=final_mean
normalizer.std = final_std
model = DiffusionModel(3,normalizer,device)
model = model.to(device)

In [12]:
import torch.optim as optim
optimizer = optim.AdamW(model.parameters(),lr=LEARNING_RATE)

In [13]:
mae_loss = nn.L1Loss()

In [15]:
ret = training_loop(EPOCHS,optimizer,model,mae_loss,train_loader,device)

Epoch loop:   0%|          | 0/50 [01:43<?, ?it/s]


RuntimeError: Given groups=1, weight of size [64, 32, 1, 1], expected input[64, 64, 64, 64] to have 32 channels, but got 64 channels instead